In [1]:
import os
import numpy as np
import sys
import torch
import time
from ctypes import *


# For jupyter
if "/home/yskim/projects/sparse-bfp" not in sys.path:
    sys.path.append("/home/yskim/projects/sparse-bfp")
os.environ["BFP_HOME"] = "/home/yskim/projects/sparse-bfp"


from util.bfp.bfp_config import BfpConfig
from util.custom_transpose import custom_transpose_2d
from util.bfp.bfp_gemm import BfpGemm
from util.reprod_util import set_reproducibility

set_reproducibility(23456)

In [51]:
lhs_ori_M = [
    '00000000111001011010011101011100', 
    '00000000100011101110010111000011', 
    '00000000110000110011101010110000', 
    '00000000100011000101101011111101', 
    '00000000101000001111000101101010', 
    '00000000101101101110110011001110', 
    '00000000100100111111001110011110', 
    '00000000110111101100001011111000', 
    '00000000100011011000101011011111', 
    '00000000100011111110000011001100', 
    '00000000101010110010100011110110', 
    '00000000110110100011101111101111', 
    '00000000110101011000011101100110', #
    '00000000100010001011101010011110', 
    '00000000101000010110111111110001', 
    '00000000100110001101000001101101'
]
rhs_ori_M = [
    '00000000110110010011000101001011', 
    '00000000101110100001010100001101', 
    '00000000100000011100010010110011', 
    '00000000111001100001111111110011', 
    '00000000111101101111010000100000', 
    '00000000111101101011110011011100', 
    '00000000100000110110100010010111', 
    '00000000100010000000101010100011', 
    '00000000110110111111100101001111', 
    '00000000111011000111111101100111', 
    '00000000100000010110100011000111', 
    '00000000101000110100110011010010', 
    '00000000111100100110010001011010', # 
    '00000000110000110110001000101010', 
    '00000000110101111101100011000111', 
    '00000000100000111101001000101100'
]

lhs_ori_E = [-21, -14, -14, -21, -20, -16, -26, -22, -14, -17, -23, -14, -21, -14, -22, -19]
rhs_ori_E = [-7, -5, -4, -6, -7, -5, -3, -4, -5, -5, -3, -6, -5, -4, -3, -4]

lhs_multi_exps = [-14, -19, -23, -2147483648]
rhs_multi_exps = [-3, -7, -2147483648, -2147483648]

lhs_E_index = [1, 0, 0, 1, 1, 0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 1]
rhs_E_index = [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

lhs_S = [-1, 1, -1,  1,  1, -1,  1,  1, -1,  1,  1, 1,  1, -1,  1, -1]
rhs_S = [ 1, 1, -1, -1, -1,  1, -1, -1,  1, -1, -1, 1, -1,  1, -1,  1]

lhs_multi_exps_M = [ 4, 9, 12, 2,  5, 3, 1, 2, 8, 1, 11, 14, 4, 9,  1, 9]
rhs_multi_exps_M = [14, 3,  4, 1, 15, 4, 8, 4, 3, 3,  9,  1, 4, 6, 14, 4]
# lhs_multi_exps_M = [
#     '0010', '0010', '0010', '1000',
#     '1010', '1001', '0110', '0101',
#     '1010', '0011', '0010', '0100',
#     '0011', '0110', '0100', '1101'
# ]
# rhs_multi_exps_M = [
#     '0101', '0011', '0111', '0101',
#     '1010', '0010', '0101', '0010',
#     '0010', '0010', '0100', '0010', 
#     '0010', '1001', '1010', '1001'
# ]

lhs_E = [-14]
rhs_E = [-3]

lhs_M = [0, 9, 12, 0, 0, 3, 0, 0, 8, 1, 0, 14, 1, 9,  0, 0]
rhs_M = [1, 3,  4, 1, 1, 4, 8, 4, 3, 3, 9,  1, 4, 6, 14, 4]
# lhs_M = [
#     '0010', '0010', '0000', '0000',
#     '1010', '0000', '0110', '0101',
#     '1010', '0011', '0010', '0100',
#     '0011', '0110', '0000', '0000'
# ]
# rhs_M = [
#     '0101', '0011', '0111', '0101',
#     '1010', '0010', '0000', '0010',
#     '0010', '0010', '0001', '0010',
#     '0010', '0001', '1010', '1001'
# ]

# print(f"{[f'{e:04b}' for e in rhs_M]}")

# -0.000001008622348 ours
# +0.000005960464478 bfp
# +0.000006123099865 answer

l = int('00000000110101011000011101100110', 2)
E_l = pow(2., -21)

r = int('00000000111100100110010001011010', 2)
E_r = pow(2., -5)

l_val = (l / pow(2., 23)) * E_l
r_val = (r / pow(2., 23)) * E_r

print(f"{-1. * (l_val * r_val):.15f}")

print(f"{(l_val):.15f} * {r_val:.15f} = {l_val * r_val:.15f}")

print("ours")
l_val = (4. / pow(2., 3)) * pow(2., -19)
r_val = (4. / pow(2., 3)) * pow(2., -3)
print(f"{(l_val):.15f} * {r_val:.15f} = {l_val * r_val:.15f}")

print("bfp")
l_val = (1. / pow(2., 3)) * pow(2., -14)
r_val = (4. / pow(2., 3)) * pow(2., -3)
print(f"{(l_val):.15f} * {r_val:.15f} = {l_val * r_val:.15f}")


-0.000000047073351
0.000000795457140 * 0.059177733957767 = 0.000000047073351
ours
0.000000953674316 * 0.062500000000000 = 0.000000059604645
bfp
0.000007629394531 * 0.062500000000000 = 0.000000476837158


In [38]:
result = 0.
for i in range(16):
    mantissa = lhs_S[i] * lhs_multi_exps_M[i] * rhs_S[i] * rhs_multi_exps_M[i]
    if mantissa < 0:
        sign = -1.
        mantissa *= -1
    else:
        sign = 1.
        mantissa *= 1
    mantissa /= pow(2., 6)
    print(f"{i:-2}: {sign:+} x {mantissa:.10f} x 2^{lhs_multi_exps[lhs_E_index[i]] + rhs_multi_exps[rhs_E_index[i]]:+}")

    result += sign * pow(2., lhs_multi_exps[lhs_E_index[i]] + rhs_multi_exps[rhs_E_index[i]]) * mantissa

# result += -1.0 * 0.0625000000 * pow(2., -17)
# result += -1.0 * 0.2500000000 * pow(2., -22)

print(f"{result:.15f}")

 0: -1.0 x 0.8750000000 x 2^-26
 1: +1.0 x 0.4218750000 x 2^-17
 2: +1.0 x 0.7500000000 x 2^-17
 3: -1.0 x 0.0312500000 x 2^-22
 4: -1.0 x 1.1718750000 x 2^-26
 5: -1.0 x 0.1875000000 x 2^-17
 6: -1.0 x 0.1250000000 x 2^-26
 7: -1.0 x 0.1250000000 x 2^-22
 8: -1.0 x 0.3750000000 x 2^-17
 9: -1.0 x 0.0468750000 x 2^-17
10: -1.0 x 1.5468750000 x 2^-26
11: +1.0 x 0.2187500000 x 2^-17
12: -1.0 x 0.2500000000 x 2^-22
13: -1.0 x 0.8437500000 x 2^-17
14: -1.0 x 0.2187500000 x 2^-22
15: -1.0 x 0.5625000000 x 2^-22
-0.000000815372914


In [54]:
result = 0.
for i in range(16):
    mantissa = lhs_S[i] * lhs_M[i] * rhs_S[i] * rhs_M[i]
    if mantissa < 0:
        sign = -1.
        mantissa *= -1
    else:
        sign = 1.
        mantissa *= 1

    mantissa /= pow(2., 6)
    # print(f"{i:-2}: {sign:+} x {mantissa:.10f} x 2^{lhs_E[0] + rhs_E[0]:+}")

    if mantissa != 0.:
        print(f"{sign:+} x {mantissa:.10f} x 2^{lhs_E[0] + rhs_E[0]:+} = {sign * pow(2., lhs_E[0] + rhs_E[0]) * mantissa:+.15f}")

    result += sign * pow(2., lhs_E[0] + rhs_E[0]) * mantissa

print(f"{result:.15f}")

# -0.000000815372914 ours
# -0.000000953674316 bfp
# -0.000000936099696 answer

+1.0 x 0.4218750000 x 2^-17 = +0.000003218650818
+1.0 x 0.7500000000 x 2^-17 = +0.000005722045898
-1.0 x 0.1875000000 x 2^-17 = -0.000001430511475
-1.0 x 0.3750000000 x 2^-17 = -0.000002861022949
-1.0 x 0.0468750000 x 2^-17 = -0.000000357627869
+1.0 x 0.2187500000 x 2^-17 = +0.000001668930054
-1.0 x 0.0625000000 x 2^-17 = -0.000000476837158
-1.0 x 0.8437500000 x 2^-17 = -0.000006437301636
-0.000000953674316


In [32]:
print(f"{-1.0 * 0.0625000000 * pow(2.,-17):.15f}")
print(f"{-1.0 * 0.2500000000 * pow(2., -22):.15f}")

-0.000000476837158
-0.000000059604645


In [63]:
fp_res = 0.
bfp_res = 0.
ours_res = 0.

for i in range(16):
    # FP
    fp_l_val = lhs_S[i] * (int(lhs_ori_M[i], 2) / pow(2., 23)) * pow(2., lhs_ori_E[i])
    fp_r_val = rhs_S[i] * (int(rhs_ori_M[i], 2) / pow(2., 23)) * pow(2., rhs_ori_E[i])
    fp_inter = fp_l_val * fp_r_val
    fp_res += fp_inter

    # BFP
    bfp_l_val = lhs_S[i] * (lhs_M[i] / pow(2., 3)) * pow(2., lhs_E[0])
    bfp_r_val = rhs_S[i] * (rhs_M[i] / pow(2., 3)) * pow(2., rhs_E[0])
    bfp_inter = bfp_l_val * bfp_r_val
    bfp_res += bfp_inter

    # ours
    ours_l_val = lhs_S[i] * (lhs_multi_exps_M[i] / pow(2., 3)) * pow(2., lhs_multi_exps[lhs_E_index[i]])
    ours_r_val = rhs_S[i] * (rhs_multi_exps_M[i] / pow(2., 3)) * pow(2., rhs_multi_exps[rhs_E_index[i]])
    ours_inter = ours_l_val * ours_r_val
    ours_res += ours_inter

    print(f"{fp_l_val:+.15f} * {fp_r_val:+.15f} = {fp_inter:+.15f} | {bfp_l_val:+.15f} * {bfp_r_val:+.15f} = {bfp_inter:+.15f} | {ours_l_val:+.15f} * {ours_r_val:+.15f} = {ours_inter:+.15f}")

print(f"FP   : {fp_res:.15f}")
print(f"BFP  : {bfp_res:.15f}")
print(f"OURS : {ours_res:.15f}")

-0.000000855526878 * +0.013256381265819 = -0.000000011341190 | -0.000000000000000 * +0.015625000000000 = -0.000000000000000 | -0.000000953674316 * +0.013671875000000 = -0.000000013038516
+0.000068138841016 * +0.045430231839418 = +0.000003095563345 | +0.000068664550781 * +0.046875000000000 = +0.000003218650818 | +0.000068664550781 * +0.046875000000000 = +0.000003218650818
-0.000093092559837 * -0.063363455235958 = +0.000005898666248 | -0.000091552734375 * -0.062500000000000 = +0.000005722045898 | -0.000091552734375 * -0.062500000000000 = +0.000005722045898
+0.000000522864696 * -0.028091406449676 = -0.000000014688005 | +0.000000000000000 * -0.015625000000000 = -0.000000000000000 | +0.000000476837158 * -0.015625000000000 = -0.000000007450581
+0.000001199118969 * -0.015072852373123 = -0.000000018074143 | +0.000000000000000 * -0.015625000000000 = -0.000000000000000 | +0.000001192092896 * -0.014648437500000 = -0.000000017462298
-0.000021806361474 * +0.060238704085350 = -0.000001313586956 | -0

In [ ]:
def load_inout(
    path_root: str, 
    phase: str, 
    layer_index: int) -> tuple[torch.Tensor, torch.Tensor]:
    path = f"{path_root}/{phase}-{layer_index}-fp"

    lhs = np.load(f"{path}-lhs.npy", allow_pickle=False)
    rhs = np.load(f"{path}-rhs.npy", allow_pickle=False)

    return (torch.tensor(lhs), torch.tensor(rhs))

def get_gemm_results(
    lhs: torch.Tensor, 
    rhs: torch.Tensor, 
    is_flex_bfp: bool) -> tuple[torch.Tensor, torch.Tensor, float, float]:

    M = lhs.shape[0]
    K = lhs.shape[1]
    N = rhs.shape[1]
    # print(f"dims: [{M}, {K}] x [{K}, {N}]")

    # if not is_flex_bfp:
    #     print(f"BFP m bits: {BfpConfig.bfp_M_Bit}")
    #     # print(f"sort size: {BfpConfig.chunk_size_to_sort}")
    # else:
    #     print("Flex BFP")

    rhs_t = torch.empty((N, K), dtype=torch.float).to('cuda')
    custom_transpose_2d(dst=rhs_t, src=rhs)

    bfp_gemm = BfpGemm(
        lhs.shape,
        rhs_t.shape,
        use_stochastic_rounding=True,
        use_flex_bfp=(True if is_flex_bfp else False)
    )

    bfp_outputs = bfp_gemm.run(lhs, rhs_t)
    fp_outputs = torch.matmul(lhs, rhs)

    lhs_E = bfp_gemm.lhs_wrapper.bfp_E.flatten().type(dtype=torch.float)
    rhs_E = bfp_gemm.rhs_wrapper.bfp_E.flatten().type(dtype=torch.float)
    # print(f"")
    # print(f"lhs mean: {torch.mean(lhs_E):02.1f}, max: {torch.max(lhs_E):02.1f}, min: {torch.min(lhs_E):02.1f}")
    # print(f"rhs mean: {torch.mean(rhs_E):02.1f}, max: {torch.max(rhs_E):02.1f}, min: {torch.min(rhs_E):02.1f}")

    lhs_bit = np.mean(bfp_gemm.lhs_wrapper.bfp_bit_level.cpu().numpy().flatten())
    rhs_bit = np.mean(bfp_gemm.rhs_wrapper.bfp_bit_level.cpu().numpy().flatten())

    return (fp_outputs, bfp_outputs, lhs_bit, rhs_bit, lhs, rhs_t, bfp_gemm)

In [ ]:
BfpConfig.bfp_M_Bit = 4
log_path = "/home/yskim/projects/sparse-bfp/resnet50-inout-log/epoch0-lr0.001"
is_flex_bfp = False
phase = "bwd-w"

for i in range(1, 53):
    lhs_ori, rhs_ori = load_inout(log_path, phase, i)
    fp_outputs, bfp_outputs, lhs_bit, rhs_bit, lhs, rhs, bfp_gemm,  = get_gemm_results(lhs_ori.to("cuda"), rhs_ori.to("cuda"), is_flex_bfp=False)
    fp_outputs = fp_outputs.cpu().numpy()
    bfp_outputs = bfp_outputs.cpu().numpy()
    lhs = lhs.cpu().numpy()
    rhs = rhs.cpu().numpy()

    dist_8bit = np.abs(fp_outputs - bfp_outputs)
    print(f"[{i:-3d}th] max: {np.max(dist_8bit):.10f}, mean: {np.mean(dist_8bit):.10f}, min: {np.min(dist_8bit):.10f}")


In [ ]:
# get_results(log_path, phase, start, end, is_flex_bfp)
BfpConfig.bfp_M_Bit = 4
log_path = "/home/yskim/projects/sparse-bfp/resnet50-inout-log/epoch0-lr0.001"
is_flex_bfp = False
phase = "fwd"

lhs_ori, rhs_ori = load_inout(log_path, phase, 40)

# print(lhs_ori.shape)
# print(rhs_ori.shape)

# lhs_ori = lhs_ori[:,0:16]
# rhs_ori = rhs_ori[0:16,:]

# lhs_ori, rhs_ori = torch.abs(lhs_ori), torch.abs(rhs_ori)

fp_outputs, bfp_outputs, lhs_bit, rhs_bit, lhs, rhs, bfp_gemm,  = get_gemm_results(lhs_ori.to("cuda"), rhs_ori.to("cuda"), is_flex_bfp=False)
fp_outputs = fp_outputs.cpu().numpy()
bfp_outputs = bfp_outputs.cpu().numpy()
lhs = lhs.cpu().numpy()
rhs = rhs.cpu().numpy()

In [ ]:
# dist_8bit = np.abs(fp_outputs - bfp_outputs)
# print(f"{np.max(dist_8bit):.15f}")
# print(f"{np.mean(dist_8bit):.15f}")
# print(f"{np.min(dist_8bit):.15f}")

# fp_outputs += 1e-15
dist_8bit = np.abs((fp_outputs - bfp_outputs) / (fp_outputs + 1e-15)) * 100.
# dist_6bit = np.abs(1. - np.abs(bfp_8bit / fp).flatten()) * 100.
# dist_4bit = np.abs(1. - np.abs(bfp_8bit / fp).flatten()) * 100.
# dist_2bit = np.abs(1. - np.abs(bfp_outputs / fp_outputs).flatten()) * 100.


#######
mask = dist_8bit >= 30.
bfp_outputs[mask] = fp_outputs[mask]
dist_8bit = np.abs((fp_outputs - bfp_outputs) / (fp_outputs + 1e-15)) * 100.
#######


print(f"{lhs.shape} x {rhs.shape} = {bfp_outputs.shape}")

outliers = []
non = []
outlier_cnt = 0
non_cnt = 0

outlier_pos = []

print(outlier_cnt)

for r in range(dist_8bit.shape[0]):
    for c in range(dist_8bit.shape[1]):
        # if dist_8bit[r, c] > 5.:
        if dist_8bit[r, c] > 10.:
            outlier_cnt += 1
            outliers.append(dist_8bit[r, c])
            outlier_pos.append((r, c))
        else:
            non_cnt += 1
            non.append(dist_8bit[r, c])

print(f"out mean: {np.mean(outliers):.15f}%")
print(f"non mean: {np.mean(non):.15f}%")
print(f"entire mean: {np.mean(non + outliers)} ({non_cnt})")
print(f"size: {dist_8bit.size}")
print(f"outlier percentage: {(outlier_cnt / (outlier_cnt+non_cnt)) * 100.:.2f}%")
print(f"min: {np.min(outliers):.15f}")
print(f"max: {np.max(outliers):.15f}")


In [ ]:
abs_error = np.abs(bfp_outputs - fp_outputs)
print(f"{np.max(abs_error):.15f}")
print(f"{np.min(abs_error):.15f}")
print(f"{np.mean(abs_error):.15f}")

In [ ]:
size = int(np.prod(dist_8bit.shape))
start = int(np.floor(size * 0.01))
end = size
sorted = np.sort(abs_error.flatten())[start:end]
# sorted = np.sort(dist_8bit.flatten())[start:end]
import matplotlib.pyplot as plt
# plt.ylim(0, 100)
plt.plot(np.arange(sorted.size), sorted)

In [ ]:
# print(outlier_pos)

# pos0 = (212, 5)
# pos1 = (384, 5)
# pos2 = (540, 5)
# pos3 = (3, 5)

pos0 = (0, 57)
pos1 = (3825, 57)
pos2 = (5864, 57)
pos3 = (123, 57)

lhs_S_0 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_S[pos0[0]].cpu()])
lhs_M_int_0 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_M[pos0[0]].cpu()])
lhs_M_0 = [f"{e.item():08b}" for e in bfp_gemm.lhs_wrapper.bfp_M[pos0[0]].cpu()]
lhs_32_bit_M_0 = [f"{e.item():024b}" for e in bfp_gemm.lhs_wrapper.bfp_32_bit_M[pos0[0]].cpu()]
lhs_E_0 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_E[pos0[0]].cpu()])

lhs_S_1 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_S[pos1[0]].cpu()])
lhs_M_int_1 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_M[pos1[0]].cpu()])
lhs_M_1 = [f"{e.item():08b}" for e in bfp_gemm.lhs_wrapper.bfp_M[pos1[0]].cpu()]
lhs_32_bit_M_1 = [f"{e.item():024b}" for e in bfp_gemm.lhs_wrapper.bfp_32_bit_M[pos1[0]].cpu()]
lhs_32_bit_M_int_1 = [e for e in bfp_gemm.lhs_wrapper.bfp_32_bit_M[pos1[0]].cpu()]
lhs_ori_E_1 = np.array([e for e in bfp_gemm.lhs_wrapper.ori_E[pos1[0]].cpu()])
lhs_E_1 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_E[pos1[0]].cpu()])
lhs_E_diff_1 = lhs_E_1 - lhs_ori_E_1

lhs_S_2 = [e for e in bfp_gemm.lhs_wrapper.bfp_S[pos2[0]].cpu()]
lhs_M_int_2 = [e for e in bfp_gemm.lhs_wrapper.bfp_M[pos2[0]].cpu()]
lhs_M_2 = [f"{e.item():08b}" for e in bfp_gemm.lhs_wrapper.bfp_M[pos2[0]].cpu()]
# lhs_ori_E_2 = [f"{e.item()}"  for e in bfp_gemm.lhs_wrapper.ori_E[pos2[0]].cpu()]
lhs_E_2 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_E[pos2[0]].cpu()])

lhs_S_3 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_S[pos3[0]].cpu()])
lhs_M_int_3 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_M[pos3[0]].cpu()])
lhs_M_3 = [f"{e.item():08b}" for e in bfp_gemm.lhs_wrapper.bfp_M[pos3[0]].cpu()]
lhs_32_bit_M_3 = [f"{e.item():024b}" for e in bfp_gemm.lhs_wrapper.bfp_32_bit_M[pos3[0]].cpu()]
lhs_32_bit_M_int_3 = [e for e in bfp_gemm.lhs_wrapper.bfp_32_bit_M[pos3[0]].cpu()]
lhs_E_3 = [e for e in bfp_gemm.lhs_wrapper.bfp_E[pos3[0]].cpu()]
lhs_ori_E_3 = np.array([e for e in bfp_gemm.lhs_wrapper.ori_E[pos3[0]].cpu()])
lhs_E_3 = np.array([e for e in bfp_gemm.lhs_wrapper.bfp_E[pos3[0]].cpu()])
lhs_E_diff_3 = lhs_E_3 - lhs_ori_E_3

rhs_S = np.array([e for e in bfp_gemm.rhs_wrapper.bfp_S[pos0[1]].cpu()])
rhs_M_int = np.array([e for e in bfp_gemm.rhs_wrapper.bfp_M[pos0[1]].cpu()])
rhs_M = [f"{e.item():08b}" for e in bfp_gemm.rhs_wrapper.bfp_M[pos0[1]].cpu()]
rhs_32_bit_M = [f"{e.item():024b}" for e in bfp_gemm.rhs_wrapper.bfp_32_bit_M[pos0[1]].cpu()]
rhs_32_bit_M_int = [e for e in bfp_gemm.rhs_wrapper.bfp_32_bit_M[pos0[1]].cpu()]
rhs_E = np.array([e for e in bfp_gemm.rhs_wrapper.bfp_E[pos0[1]].cpu()])
rhs_ori_E = np.array([e for e in bfp_gemm.rhs_wrapper.ori_E[pos0[1]].cpu()])

# print(lhs_E_diff_2)

# print(np.sum(np.array(lhs_S_0) * np.array(rhs_S)))
# print(np.sum(np.array(lhs_S_1) * np.array(rhs_S)))
# print(np.sum(np.array(lhs_S_3) * np.array(rhs_S)))
# print(np.sum(np.array(lhs_S) ))

# for i in range(len(lhs_M_0)):
#     print(
#         f"lhs0: {lhs_S_0[i]:+} x {lhs_M_0[i]} x {lhs_E_0[i]} | "
#         f"lhs1: {lhs_S_1[i]:+} x {lhs_M_1[i]} x {lhs_E_1[i]} | "
#         f"lhs2: {lhs_S_2[i]:+} x {lhs_M_2[i]} x {lhs_E_2[i]}({lhs_E_diff_2[i]:2d}) | "
#         f"lhs3: {lhs_S_3[i]:+} x {lhs_M_3[i]} x {lhs_E_3[i]}({lhs_E_diff_3[i]:2d}) || "
#         f"rhs: {lhs_S[i]:+} x {rhs_M[i]} x {rhs_E[i]}")

# for i in range(len(lhs_M_0)):
#     print(
#         f"high : {lhs_S_1[i]:+} x {lhs_M_1[i]} ({lhs_32_bit_M_1[i]}) x {lhs_E_1[i]} ({lhs_E_diff_1[i]:-2d}) | "
#         f"rhs: {rhs_S[i]:+} x {rhs_M[i]} x {rhs_E[i]} || "
#         f"low  : {lhs_S_3[i]:+} x {lhs_M_3[i]} ({lhs_32_bit_M_3[i]}) x {lhs_E_3[i]} ({lhs_E_diff_3[i]:-2d}) | "
#         f"rhs: {rhs_S[i]:+} x {rhs_M[i]} ({rhs_32_bit_M[i]}) x {rhs_E[i]}")

In [ ]:
print("FP")
print(f"{fp_outputs[pos1]:+.15f}")
print(f"{fp_outputs[pos3]:+.15f}")
print()

print("BFP")
print(f"{bfp_outputs[pos1]:+.15f}")
print(f"{bfp_outputs[pos3]:+.15f}")
print()

print(dist_8bit[pos1])
print(dist_8bit[pos3])

In [ ]:
from re import M


def dp(lhs_S, lhs_M, lhs_E, rhs_S, rhs_M, rhs_E, bfp_m_bits):
    # m_sum = np.dot((lhs_S * lhs_M), (rhs_S * rhs_M))
    # E_sum = lhs_E[0] + rhs_E[0]
    # print(lhs_E[0])
    # print(rhs_E[0])

    # if m_sum >= 0:
    #     sign = 1.
    #     m_sum = 1. * m_sum 
    # else:
    #     sign = -1.
    #     m_sum = -1. * m_sum

    # m_sum /= pow(2., 2 * (bfp_m_bits - 1))

    # return sign * pow(2., E_sum) * m_sum
    results = []
    result = 0.

    nz_sum = 0.

    for i in range(16):
        m_sum = (np.int64(lhs_S[i]) * np.int64(lhs_M[i])) * (np.int64(rhs_S[i]) * np.int64(rhs_M[i]))

        E_sum = lhs_E[0] + rhs_E[0]

        if m_sum >= 0:
            sign = 1.
            m_sum = 1. * m_sum 
        else:
            sign = -1.
            m_sum = -1. * m_sum

        m_sum /= pow(2., 2 * (bfp_m_bits - 1))

        value = sign * pow(2., E_sum) * m_sum

        # print(f"{value:+.15f}")

        results.append(value)

        result += value
    print(f"nz sum: {nz_sum:+.15f}")

    return result, results



def dp_fp(lhs_S, lhs_M, lhs_E, rhs_S, rhs_M, rhs_E, bfp_m_bits):
    results = []
    result = 0.

    for i in range(16):
        m_sum = (np.int64(lhs_S[i]) * np.int64(lhs_M[i])) * (np.int64(rhs_S[i]) * np.int64(rhs_M[i]))
        E_sum = lhs_E[i] + rhs_E[i]

        if m_sum >= 0:
            sign = 1.
            m_sum = 1. * m_sum 
        else:
            sign = -1.
            m_sum = -1. * m_sum

        m_sum /= pow(2., 2 * (bfp_m_bits - 1))

        value = sign * pow(2., E_sum) * m_sum

        # print(f"{value:+.15f}")

        results.append(value)

        result += value

    return result, results

bfp_result, bfp_results = dp(lhs_S_1, lhs_M_int_1, lhs_E_1, rhs_S, rhs_M_int, rhs_E, 8)
fp_result, fp_results = dp_fp(lhs_S_1, lhs_32_bit_M_int_1, lhs_ori_E_1, rhs_S, rhs_32_bit_M_int, rhs_ori_E, 24)

print(f"BFP: {bfp_result:+.15f}")
print(f"FP:  {fp_result:+.15f}")
print(f"ERR: {np.abs(1. - np.abs(bfp_result / fp_result) * 100.):+.2f}%")
# print(f"{bfp_outputs[pos1]:+.15f}")
print()

# for i in range(16):
    # print(f"{i:03d} bfp: {bfp_results[i]:+.15f} | fp: {fp_results[i]:+.15f}")

# print(f"SUM bfp: {bfp_result:+.15f} | fp: {fp_result:+.15f}")

print("[LOG]")
for i in range(len(lhs_M_0)):
    print(
        f"{i:-3d} "
        f"rhs : {lhs_S_1[i]:+} x {lhs_M_1[i]} x {lhs_E_1[i]} (E diff.: {lhs_E_diff_1[i]:-2d}) | "
        # f"rhs: {rhs_S[i]:+} x {rhs_M[i]} x {rhs_E[i]} || "
        # f"low  : {lhs_S_3[i]:+} x {lhs_M_3[i]} x {lhs_E_3[i]} ({lhs_E_diff_3[i]:-2d}) | "
        f"rhs: {rhs_S[i]:+} x {rhs_M[i]} x {rhs_E[i]} || "
        f"[{i:-3d}th MUL] bfp: {bfp_results[i]:+.15f} | fp: {fp_results[i]:+.15f}")

# 8 exponent
# quantization -> 32-bit 8-bit, 4-bit / 
# block floating point /
# floating point
# 2^3 / 1.001

In [ ]:
# 하나의 오차가 차지하는 영향이 커진다
# 부호가 하나인 경우, 한 쪽으로 accumulate되기 때문에 오차의 양이 acc된 값들에 비해서 적을텐데
# 음양이 둘 다 들어가는 경우, acc된 값이 "작아질 수 있으니", 같은 양의 오차가 차지하는 비율이 커진다
# 5, 6, 8, 14
print(f"BFP: {bfp_result:+.15f}")
print(f"FP:  {fp_result:+.15f}")
print(f"ZER: {fp_results[5] + fp_results[6] + fp_results[8] + fp_results[14]:+.15f}")

bfp_res = 0.
fp_res = 0.

for i in range(16):
    if i in [5, 6, 8, 14]:
        continue

    bfp_res += bfp_results[i]
    fp_res += fp_results[i]

print()
print(f"BFP: {bfp_res:+.15f}")
print(f"FP:  {fp_res:+.15f}")
print(f"ERROR: {np.abs(1. - np.abs(bfp_res / fp_res) * 100):+.2f}")